# The objectives of this jupyter notebook (flood_geemap_visualisation_tool.ipynb):

1. Generation of an interactive html map, for better user understanding and presentation, selection of regions for modelling and application, collection of data points, database of actual flood map, etc.

**Note1: When you try to re-run the code below, be sure to redefine the path to the data file to suit your current environment.**

**Note2: It is recommended to use Google colab to run the following code, making sure you have permission to use it from Google Earth Engine.**

### Import Some necessary packages and autheticate the earthengine to be ready.

In [1]:
# Install necessary packages:
!pip install geemap
!earthengine authenticate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 36.3 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11122 sha256=cadbcd1b510fcd5f629a2473172c4de0533960d7af0045760eab961c0c67dba4
  Stored in directory: /root/.cache/pip/wheels/04/24/fe/140a94a7f1036003ede94579e6b4227

In [2]:
# Imports all necessary packages:
from google.colab import drive
drive.mount('/content/gdrive')
import pandas as pd
import geemap
import ee
import json
import sys
tool_folder_dir = "/content/gdrive/MyDrive/irp_project_111/tools"
sys.path.append(tool_folder_dir)
from geemap_vis_tool import create_boundary_mask, display_data_on_map, load_flood_data_from_csv
from common_fucs import convert_to_ee_rectangle

Mounted at /content/gdrive


### For Objective 1: Generation of an interactive html map for better understanding and visualisation.

In [3]:
# 1. get all the floods id that happend in Ghana,
ghana_flood_df = pd.read_csv('/content/gdrive/MyDrive/irp_project_111/data/Historical_Ghana_Flood_info_table.csv')
ghana_flood_id_ls = ghana_flood_df['id'].tolist()
print(f"For whole ghana all the floods(id) are: {ghana_flood_id_ls}")
print('-----------------------------------------------------------------------')
# 2. read critical boundaries on this project, they all have been saved in data folder:
ROI_file_path = '/content/gdrive/MyDrive/irp_project_111/data/ROI_boundaries_for_Modeling.json'
#   2.1. Open and load ROI_boundaries_for_Modeling.json file
with open(ROI_file_path, 'r') as file:
    modeling_roi = json.load(file)
ROI1_for_Modeling = modeling_roi['ROI_boundary1']
ROI2_for_Modeling = modeling_roi['ROI_boundary2']
print(f"For Modeling the 1st region: {ROI1_for_Modeling}")
print(f"For Modeling the 2nd region: {ROI2_for_Modeling}")
#   2.2. Open and load ROI_boundaries_for_Application.json file
sub_ROI_file_path = '/content/gdrive/MyDrive/irp_project_111/data/ROI_boundaries_for_Application.json'
with open(sub_ROI_file_path, 'r') as file:
    application_roi = json.load(file)
sub_ROI1_for_Application = application_roi['sub_ROI1_for_Application']
sub_ROI2_for_Application = application_roi['sub_ROI2_for_Application']
print(f"For Application the 1st region: {sub_ROI1_for_Application}")
print(f"For Application the 2nd region: {sub_ROI2_for_Application}")
print('-----------------------------------------------------------------------')
# 3. read the common flood ids that are affected both Modeling region:
common_flood_ids_file_path = '/content/gdrive/MyDrive/irp_project_111/data/common_flood_ids_for_both_Modeling_ROI.json'
#   3.1. Open and load the common_flood_ids_for_both_Modeling_ROI.json file
with open(common_flood_ids_file_path, 'r') as file:
    common_flood_ids = json.load(file)
common_flood_ids = common_flood_ids['common_flood_ids_for_both_Modeling_ROI']
print(f'Only these 6 flood (ids) will affect both Modeling ROIs: {common_flood_ids}')

# 4. plot flood map:
flood_coors_df_for_vis = load_flood_data_from_csv("/content/gdrive/MyDrive/irp_project_111/data/Modeling_Data/modeling_data.csv")
# 5. start ploting a map:
ee.Initialize()
#   5.1. define layer's viz params
flooded_and_not_perm_water_viz_params = {
    'min': 0,
    'max': 1,
    'palette': ['#CACACA', 'red']
}
jrc_perm_water_viz_params = {
    'min': 0,
    'max': 1,
    'palette': ['#CACACA', 'blue']}
duration_viz_params = {
    'min': 0,
    'max': 56,
    'palette': ['#CACACA', '#FED976', '#FD8D3C', '#FC4E2A', '#BD0026']
}
#   5.2. define each necessary boundaries:
mask_dic = {}
#       5.2.1. for ghana boundary:
ghana_boundary = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_na', 'Ghana'))
mask_dic["Ghana Boundary"] = create_boundary_mask(ghana_boundary)
#       5.2.2. for 2 modeling boundaries:
ROI_boundary1_for_Modeling = ee.Geometry.Rectangle(ROI1_for_Modeling)
ROI_boundary2_for_Modeling = ee.Geometry.Rectangle(ROI2_for_Modeling)
mask_dic["Modeling ROI1"] = create_boundary_mask(ROI_boundary1_for_Modeling)
mask_dic["Modeling ROI2"] = create_boundary_mask(ROI_boundary2_for_Modeling)
subROI_boundary1_for_Application = convert_to_ee_rectangle(sub_ROI1_for_Application)
subROI_boundary2_for_Application = convert_to_ee_rectangle(sub_ROI2_for_Application)
mask_dic["Application sub-ROI1"] = create_boundary_mask(subROI_boundary1_for_Application)
mask_dic["Application sub-ROI2"] = create_boundary_mask(subROI_boundary2_for_Application)
#   5.3. save the final map to .html file:
save_html_map_path = "/content/gdrive/MyDrive/irp_project_111/results/interactive_gee_flood_map.html"
map = display_data_on_map(common_flood_ids, flooded_and_not_perm_water_viz_params, jrc_perm_water_viz_params, duration_viz_params, mask_dic, flood_coors_df_for_vis, save_html_map_path)

For whole ghana all the floods(id) are: [2320, 2576, 2947, 3166, 3151, 3534, 3555, 3663, 3747, 3872, 4683]
-----------------------------------------------------------------------
For Modeling the 1st region: [-0.87, 8.05, -0.37, 8.55]
For Modeling the 2nd region: [-1.2, 9.7, -0.9, 10.0]
For Application the 1st region: [[[-0.642576, 8.172062], [-0.502478, 8.172062], [-0.502478, 8.339226], [-0.642576, 8.339226], [-0.642576, 8.172062]]]
For Application the 2nd region: [[[-1.011302, 9.744245], [-0.929578, 9.744245], [-0.929578, 9.856564], [-1.011302, 9.856564], [-1.011302, 9.744245]]]
-----------------------------------------------------------------------
Only these 6 flood (ids) will affect both Modeling ROIs: [2320, 3166, 3534, 3663, 3747, 4683]


Map(center=[7.9465, -1.0232], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

In [ ]:
# 6.(optional) Get the coordinates of the points and region drawn on the when the map has been shown up.
# But notice, it will only return your last coordinates of the points and region drawn from the map.
if map.user_roi is not None:
    try:
        result = map.user_roi.getInfo()['coordinates']
        print(result)
    except Exception as e:
        print("Failed to get polygon coordinates:", e)
else:
    print('Please draw a polygon on the map.')

[[[358.450928, 7.491835], [361.175537, 7.491835], [361.175537, 8.753437], [358.450928, 8.753437], [358.450928, 7.491835]]]
